The outline of the project is as follows:
1. Import the required libraries.
2. Download the historical stock prices using Yahoo Finance API.
3. Calculate the daily returns of each stock and the market.
4. Compute the mean daily return and covariance matrix.
5. Define the mean-variance optimization function and solve for the optimal portfolio weights.
6. Print the results.
7. Calculate the Sharpe ratio
8. Solve for and print volatility

In [1]:
import pandas as pd
import numpy as np
from pandas_datareader import data as pdr
import yfinance as yf
from scipy.optimize import minimize
yf.pdr_override()

We use the Pandas library to create an empty DataFrame object called df. We then loop over each ticker symbol in the tickers list (JNJ, PG, PFE, and SPY) and download the adjusted close prices for that symbol using the get_data_yahoo() method from the Pandas DataReader (pdr) library. The start and end dates of the historical data to download are specified by start_date and end_date.

The downloaded data is then assigned to a new column in the DataFrame df with the same name as the ticker symbol. This creates a DataFrame with 4 columns, one for each ticker symbol, and rows containing the daily adjusted close prices of each stock between the start and end dates.

Overall, this code downloads historical stock price data for a specific date range for multiple companies and stores it in a Pandas DataFrame.

In [2]:
tickers = ['JNJ', 'PG', 'PFE']
start_date = '2020-03-01'
end_date = '2022-03-01'
df = pd.DataFrame()
for ticker in tickers:
    data = pdr.get_data_yahoo(ticker, start_date, end_date)['Adj Close']
    df[ticker] = data

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


We calculate the daily percentage change in the adjusted close prices of multiple stocks and stores the result in a new DataFrame called returns. The pct_change() method computes the percentage change between the current and a prior element in a DataFrame, and dropna() method removes any resulting missing values.

In [3]:
returns = df.pct_change().dropna()

Now, we compute the mean daily return and the covariance matrix of the daily returns for the stocks in the returns DataFrame calculated in the previous step. The mean() method computes the mean daily return for each stock, and the cov() method computes the covariance matrix of the daily returns for all stocks. The resulting mu and Sigma variables are used in Step 5 to optimize the portfolio.

In [4]:
mu = returns.mean()
Sigma = returns.cov()

We define a Python function called mean_variance_optimization that calculates the optimal portfolio weights that minimize portfolio risk for a given expected return, based on the mean daily returns and covariance matrix of multiple stocks. It uses linear algebra operations to calculate the optimal weights, which are returned as a NumPy array.

In [5]:
def mean_variance_optimization(mu, Sigma):
    n = len(mu)
    A = np.ones(n)
    b = 1
    C = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            C[i, j] = 2 * Sigma.iloc[i, j]
    w = np.linalg.inv(C) @ A / (A.T @ np.linalg.inv(C) @ A)
    return w

Now, we solve for the optimal portfolio weights.


In [6]:
w = mean_variance_optimization(mu, Sigma)

Now, we print the results.

In [7]:
print('Optimal Portfolio Weights:\n', w)
print('Expected Daily Return:', round(w @ mu, 4))
print('Portfolio Variance:', round(w @ Sigma @ w.T, 4))
print('\nCovariance Matrix:')
print(Sigma)

Optimal Portfolio Weights:
 [0.48870062 0.37878346 0.13251592]
Expected Daily Return: 0.0007
Portfolio Variance: 0.0002

Covariance Matrix:
          JNJ        PG       PFE
JNJ  0.000224  0.000175  0.000179
PG   0.000175  0.000247  0.000157
PFE  0.000179  0.000157  0.000396


The results show the optimal portfolio weights, expected daily return, and portfolio variance calculated using the mean-variance optimization function. The optimal portfolio weights represent the fraction of the total investment to allocate to each stock in the portfolio.

In this case, the optimal portfolio weights suggest investing 33.8% of the portfolio in Johnson & Johnson, 25.9% in Procter & Gamble, 10.3% in Pfizer, and 29.9% in SPDR S&P 500 ETF Trust. The expected daily return of the portfolio is 0.0007 or 0.07%, and the portfolio variance is 0.0002 or 0.02%.

These results indicate that the portfolio is expected to generate a small daily return with relatively low risk. However, it's important to note that these results are based on historical data and may not necessarily predict future performance. Therefore, investors should exercise caution and perform their own due diligence before making any investment decisions.

Now, we print the covariance matrix.

In [8]:
print('\nCovariance Matrix:')
print(Sigma)


Covariance Matrix:
          JNJ        PG       PFE
JNJ  0.000224  0.000175  0.000179
PG   0.000175  0.000247  0.000157
PFE  0.000179  0.000157  0.000396


The results show the covariance matrix of the daily returns for the four stocks included in the analysis (Johnson & Johnson, Procter & Gamble, Pfizer, and SPDR S&P 500 ETF Trust). The covariance matrix measures the degree to which the returns of the stocks move together over time.

In this case, the diagonal elements of the covariance matrix represent the variances of the individual stocks, while the off-diagonal elements represent the covariances between pairs of stocks. The results suggest that all the stocks have positive covariances, meaning that they tend to move in the same direction. However, the magnitude of the covariances varies, with some pairs of stocks having stronger correlations than others.

For example, the covariance between Johnson & Johnson and Procter & Gamble is 0.000175, while the covariance between Pfizer and SPDR S&P 500 ETF Trust is 0.000152. These results can be useful in identifying diversification opportunities and constructing optimal portfolios that balance risk and return.

Next, we calculate and print the Sharpe ratio.

In [9]:
risk_free_rate = 0.0
expected_return = w @ mu
portfolio_volatility = np.sqrt(w @ Sigma @ w.T)
sharpe_ratio = (expected_return - risk_free_rate) / portfolio_volatility
print('\nSharpe Ratio:', round(sharpe_ratio, 4))


Sharpe Ratio: 0.0483


The Sharpe ratio of 0.0544 suggests that the portfolio has generated a relatively small excess return per unit of risk taken. While positive, the Sharpe ratio is not particularly high, indicating that the portfolio may not be generating sufficient returns to justify the level of risk taken.

Lastly, we generate the portfolio volatility.

In [10]:
print('Portfolio Volatility:', round(portfolio_volatility, 4))

Portfolio Volatility: 0.0141


The portfolio volatility of 0.0137 suggests that the portfolio's returns have been relatively stable over time.

Now, we store the Tau values.

In [11]:
tau_values = [0.2, 0.5, 0.8]

We are defining a variable called "views" which is a list of tuples, where each tuple contains a Pandas DataFrame, a Pandas Series, and a Numpy array. These tuples represent different views of the same data. Each view has its own set of weights, which are stored in a Numpy array called "omega_values".

In [20]:
views = [(pd.DataFrame([[1, 0, 0], [0, 1, 0]], index=['View 1', 'View 2'], columns=['A', 'B', 'C']), 
          pd.Series([0.015, 0.04], index=['View 1', 'View 2']), np.array([0.5, 0.5])),
         (pd.DataFrame([[0, 1, -1]], index=['View 3'], columns=['A', 'B', 'C']), 
          pd.Series([0.005], index=['View 3']), np.array([0.7])),
         (pd.DataFrame([[0, 1, 0]], index=['View 4'], columns=['A', 'B', 'C']), 
          pd.Series([0.02], index=['View 4']), np.array([0.3]))]
omega_values = [0.3, 0.5, 0.7]

We are calculating the market equilibrium portfolio weights for each value of tau by first computing the inverse of the covariance matrix Sigma plus tau times the identity matrix. Then, we multiply this inverse by the expected returns vector mu to obtain mu_tilde. Finally, we calculate the market equilibrium portfolio weights pi by dividing mu_tilde by tau.

In [17]:
for tau in tau_values:
    sigma_tilde = np.linalg.inv(np.linalg.inv(Sigma) + tau*np.eye(Sigma.shape[0]))
    mu_tilde = np.dot(sigma_tilde,mu)
    pi = mu_tilde / tau 

Now, we calculate the Black-Litterman optimal portfolio for each view and omega value. We start by initializing an empty list called table_data. We then loop through each view and its corresponding omega value, and calculate the posterior sigma and posterior mu. Using these values and the market equilibrium portfolio weights, we use the minimize function to optimize the Black-Litterman weights, subject to the constraint that the sum of weights is equal to 1. We then append the tau value, omega value, and the resulting Black-Litterman weights to the table_data list.

In [18]:
table_data = []
for i, (P, Q, omega) in enumerate(views):
    omega = np.diag(np.repeat(omega_values[i], P.shape[0]))
    tau_sigma = tau * Sigma
    posterior_sigma = np.linalg.inv(np.linalg.inv(tau_sigma) + np.dot(np.dot(np.transpose(P), np.linalg.inv(omega)), P))
    posterior_mu = np.dot(posterior_sigma, (np.dot(np.dot(np.transpose(P), np.linalg.inv(omega)), Q) + np.dot(np.linalg.inv(tau_sigma), pi)))
    bl_weights = minimize(lambda w: np.dot(np.dot(w.T, posterior_sigma), w), x0=np.random.rand(mu.shape[0]), 
                              constraints=[{'type': 'eq', 'fun': lambda w: np.sum(w) - 1}], bounds=[(0,1) for i in range(mu.shape[0])])['x']
    table_data.append([tau_values[i], omega_values[i], bl_weights])

We are printing the table of Black-Litterman optimal portfolio weights for each view and omega value. We create a table with columns 'tau', 'omega', and 'weights[JNJ][PG][PFE]'. We then populate the table with the data stored in 'table_data' and print the resulting DataFrame.

In [19]:
table_columns = ['tau', 'omega', 'weights[JNJ][PG][PFE]']
df = pd.DataFrame(table_data, columns=table_columns)
print(df)

   tau  omega                              weights[JNJ][PG][PFE]
0  0.2    0.3  [0.07943659713369094, 0.5992855480232255, 0.32...
1  0.5    0.5  [0.5487967046947977, 2.7755575615628914e-17, 0...
2  0.8    0.7  [0.32983172534847516, 0.10638420506253515, 0.5...
